# به نام خدا

### تشخیص سه مربع و تشخیص درست جهت

In [9]:
import cv2
import numpy as np
import math

VIDEO_PATH = r"D:\FiraCUP\Soraat\PishVaDor\road.mp4"
DISPLAY_SCALE = 0.7

# ===== حافظه =====
prev_boxes_x = [None, None, None]  # چپ، وسط، راست

ALPHA = 0.25
MAX_JUMP = 80

# ================= توابع کمکی =================
def draw_square(img, center, size=14, color=(0,255,255)):
    x, y = center
    cv2.rectangle(img, (x-size, y-size),
                  (x+size, y+size), color, 2)

def smooth(prev, new):
    if prev is None:
        return new
    if abs(new - prev) > MAX_JUMP:
        return prev
    return int(ALPHA * new + (1 - ALPHA) * prev)

# ================= پردازش فریم =================
def process_frame(frame):
    global prev_boxes_x

    h, w = frame.shape[:2]
    car_x = w // 2
    y_ref = int(h * 0.72)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    mask = np.zeros_like(edges)
    roi = np.array([[ (0,h),(w,h),(w,int(h*0.6)),(0,int(h*0.6)) ]])
    cv2.fillPoly(mask, roi, 255)
    edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(
        edges, 1, np.pi/180,
        threshold=50,
        minLineLength=50,
        maxLineGap=140
    )

    xs = []

    # ===== جمع‌آوری نقاط X خطوط =====
    if lines is not None:
        for l in lines:
            x1,y1,x2,y2 = l[0]
            if abs(x2-x1) < 10:
                continue

            slope = (y2-y1)/(x2-x1)
            if abs(slope) < 0.3 or abs(slope) > 2:
                continue

            x_at_y = int(x1 + (y_ref-y1)*(x2-x1)/(y2-y1))
            if 0 < x_at_y < w:
                xs.append(x_at_y)

            cv2.line(frame,(x1,y1),(x2,y2),(100,100,255),1)

    if len(xs) < 10:
        return frame

    # ===== هیستوگرام تراکم =====
    hist = np.histogram(xs, bins=60, range=(0,w))[0]
    peaks = np.argsort(hist)[-3:]  # سه پیک بزرگ

    peaks = sorted(peaks)
    boxes_x = []

    for i, p in enumerate(peaks):
        x = int((p + 0.5) * (w / 60))
        x = smooth(prev_boxes_x[i], x)
        boxes_x.append(x)

    prev_boxes_x = boxes_x

    # ===== رسم سه مربع =====
    colors = [(255,0,0),(0,255,255),(0,255,0)]
    for i, x in enumerate(boxes_x):
        draw_square(frame, (x, y_ref), color=colors[i])

    # ===== تشخیص لاین =====
    center_x = boxes_x[1]
    lane = "راست" if center_x < car_x else "چپ"

    # ===== انتخاب دو مربع فعال =====
    if lane == "راست":
        active = (boxes_x[1], boxes_x[2])
    else:
        active = (boxes_x[0], boxes_x[1])

    steer_x = (active[0] + active[1]) // 2

    # ===== فلش =====
    cv2.arrowedLine(frame,
                    (car_x, h),
                    (steer_x, y_ref),
                    (0,255,0), 3)

    cv2.putText(frame,
        f"لاین: {lane}",
        (40,50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9,(0,255,0),2)

    return frame

# ================= اجرای ویدیو =================
cap = cv2.VideoCapture(VIDEO_PATH)
print("🎥 Lane Density Tracking (q خروج)")

while True:
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    processed = process_frame(frame)
    display = cv2.resize(processed, None,
                          fx=DISPLAY_SCALE,
                          fy=DISPLAY_SCALE)

    cv2.imshow("Stable Lane Squares", display)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎥 Lane Density Tracking (q خروج)


KeyboardInterrupt: 

### کد اصلی  تا اینجا

In [13]:
import cv2
import numpy as np
import math

VIDEO_PATH = r"D:\FiraCUP\Soraat\PishVaDor\road.mp4"
DISPLAY_SCALE = 0.7

# ===== حافظه =====
prev_boxes_x = [None, None, None]  # چپ، وسط، راست

ALPHA = 0.25
MAX_JUMP = 80

# ================= توابع کمکی =================
def draw_square(img, center, size=14, color=(0,255,255)):
    x, y = center
    cv2.rectangle(img, (x-size, y-size),
                  (x+size, y+size), color, 2)

def smooth(prev, new):
    if prev is None:
        return new
    if abs(new - prev) > MAX_JUMP:
        return prev
    return int(ALPHA * new + (1 - ALPHA) * prev)

# ===== محاسبه زاویه نسبت به خط عمودی =====
def angle_between_vertical(p1, p2):
    dx = p2[0] - p1[0]
    dy = p1[1] - p2[1]   # چون محور y برعکس است
    angle = math.degrees(math.atan2(dx, dy))
    return angle

# ================= پردازش فریم =================
def process_frame(frame):
    global prev_boxes_x

    h, w = frame.shape[:2]
    car_x = w // 2
    y_ref = int(h * 0.72)

    car_point = (car_x, h)
    vertical_ref = (car_x, int(h * 0.6))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    mask = np.zeros_like(edges)
    roi = np.array([[ (0,h),(w,h),(w,int(h*0.6)),(0,int(h*0.6)) ]])
    cv2.fillPoly(mask, roi, 255)
    edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(
        edges, 1, np.pi/180,
        threshold=50,
        minLineLength=50,
        maxLineGap=140
    )

    xs = []

    # ===== جمع‌آوری نقاط X خطوط =====
    if lines is not None:
        for l in lines:
            x1,y1,x2,y2 = l[0]
            if abs(x2-x1) < 10:
                continue

            slope = (y2-y1)/(x2-x1)
            if abs(slope) < 0.3 or abs(slope) > 2:
                continue

            x_at_y = int(x1 + (y_ref-y1)*(x2-x1)/(y2-y1))
            if 0 < x_at_y < w:
                xs.append(x_at_y)

            cv2.line(frame,(x1,y1),(x2,y2),(100,100,255),1)

    if len(xs) < 10:
        return frame

    # ===== هیستوگرام تراکم =====
    hist = np.histogram(xs, bins=60, range=(0,w))[0]
    peaks = np.argsort(hist)[-3:]
    peaks = sorted(peaks)

    boxes_x = []
    for i, p in enumerate(peaks):
        x = int((p + 0.5) * (w / 60))
        x = smooth(prev_boxes_x[i], x)
        boxes_x.append(x)

    prev_boxes_x = boxes_x

    # ===== رسم سه مربع =====
    colors = [(255,0,0),(0,255,255),(0,255,0)]
    for i, x in enumerate(boxes_x):
        draw_square(frame, (x, y_ref), color=colors[i])

    # ===== تشخیص لاین طبق قانون شما =====
    left_count = sum(1 for x in boxes_x if x < car_x)
    right_count = sum(1 for x in boxes_x if x > car_x)

    if left_count == 2 and right_count == 1:
        lane = "راست"
        active = (boxes_x[1], boxes_x[2])
    elif right_count == 2 and left_count == 1:
        lane = "چپ"
        active = (boxes_x[0], boxes_x[1])
    else:
        lane = "نامشخص"
        active = (boxes_x[0], boxes_x[2])

    steer_x = (active[0] + active[1]) // 2
    steer_point = (steer_x, y_ref)

    # ===== فلش =====
    cv2.arrowedLine(frame,
                    car_point,
                    steer_point,
                    (0,255,0), 3)

    # ===== خط عمودی مرجع =====
    cv2.line(frame,
             car_point,
             vertical_ref,
             (255,255,255), 2)

    # ===== زاویه =====
    angle = angle_between_vertical(car_point, steer_point)

    # ===== نمایش =====
    cv2.putText(frame,
        f"Lane: {lane}",
        (40,50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9,(0,255,0),2)

    cv2.putText(frame,
        f"Angle: {angle:+.1f}",
        (40,90),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9,(255,255,255),2)

    # ===== چاپ در ترمینال =====
    print(f"Angle: {angle:+.2f} deg | Lane: {lane}")

    return frame

# ================= اجرای ویدیو =================
cap = cv2.VideoCapture(VIDEO_PATH)
print("🎥 Lane Density Tracking (q خروج)")

while True:
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    processed = process_frame(frame)
    display = cv2.resize(processed, None,
                          fx=DISPLAY_SCALE,
                          fy=DISPLAY_SCALE)

    cv2.imshow("Stable Lane Squares", display)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎥 Lane Density Tracking (q خروج)
Angle: +6.03 deg | Lane: راست
Angle: +6.78 deg | Lane: راست
Angle: +7.33 deg | Lane: راست
Angle: +7.71 deg | Lane: راست
Angle: +7.89 deg | Lane: راست
Angle: +8.08 deg | Lane: راست
Angle: +8.26 deg | Lane: راست
Angle: +8.45 deg | Lane: راست
Angle: +8.45 deg | Lane: راست
Angle: +9.37 deg | Lane: راست
Angle: +8.63 deg | Lane: راست
Angle: +8.82 deg | Lane: راست
Angle: +9.55 deg | Lane: راست
Angle: +10.11 deg | Lane: راست
Angle: +10.47 deg | Lane: راست
Angle: +10.47 deg | Lane: راست
Angle: +10.65 deg | Lane: راست
Angle: +10.84 deg | Lane: راست
Angle: +10.29 deg | Lane: راست
Angle: +10.47 deg | Lane: راست
Angle: +10.11 deg | Lane: راست
Angle: +10.11 deg | Lane: راست
Angle: +10.11 deg | Lane: راست
Angle: +10.11 deg | Lane: راست
Angle: +10.11 deg | Lane: راست
Angle: +10.47 deg | Lane: راست
Angle: +10.65 deg | Lane: راست
Angle: +10.84 deg | Lane: راست
Angle: +11.02 deg | Lane: راست
Angle: +11.20 deg | Lane: راست
Angle: +11.20 deg | Lane: راست
Angle: +11.38 deg |

KeyboardInterrupt: 

### تشخیص مانع

In [ ]:
import cv2
import numpy as np

CHANGE_LANE_THRESHOLD = 450

current_lane = "center"
original_lane = "center"
lane_changed = False


def process_frame(frame, lane, boxes_x, y_ref):
    global current_lane, original_lane, lane_changed

    output = frame.copy()

    # ================== 1) تشخیص مانع (عین کد خودت) ==================
    orange_mask = detect_orange_obstacles(frame)
    contour, area = find_largest_obstacle(orange_mask)
    obstacle_detected = area >= CHANGE_LANE_THRESHOLD

    obstacle_in_our_lane = False
    cx = None

    if contour is not None:
        cv2.drawContours(output, [contour], -1, (0,255,0), 2)

        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            cv2.circle(output, (cx, cy), 5, (255,0,0), -1)
            cv2.putText(output, f"Area:{int(area)}", (cx-40, cy-20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    # ================== 2) تشخیص اینکه مانع در لاین ماست ==================
    if obstacle_detected and cx is not None:
        frame_center = frame.shape[1] // 2

        if current_lane == "left" and cx < frame_center:
            obstacle_in_our_lane = True
        elif current_lane == "right" and cx > frame_center:
            obstacle_in_our_lane = True

    # ================== 3) تصمیم تغییر لاین ==================
    steer_x = None

    if obstacle_detected and obstacle_in_our_lane and not lane_changed:
        # ثبت لاین اولیه
        original_lane = current_lane

        # تغییر لاین
        if current_lane == "left":
            current_lane = "right"
            steer_x = (boxes_x[1] + boxes_x[2]) // 2
        else:
            current_lane = "left"
            steer_x = (boxes_x[0] + boxes_x[1]) // 2

        lane_changed = True

    # ================== 4) بازگشت به لاین اصلی ==================
    elif lane_changed and not obstacle_detected:
        current_lane = original_lane
        lane_changed = False

        if current_lane == "left":
            steer_x = (boxes_x[0] + boxes_x[1]) // 2
        else:
            steer_x = (boxes_x[1] + boxes_x[2]) // 2

    # ================== 5) حالت عادی (بدون مانع) ==================
    if steer_x is None:
        if current_lane == "left":
            steer_x = (boxes_x[0] + boxes_x[1]) // 2
        else:
            steer_x = (boxes_x[1] + boxes_x[2]) // 2

    steer_point = (steer_x, y_ref)

    # ================== 6) رسم فلش ==================
    cv2.arrowedLine(
        output,
        (frame.shape[1] // 2, frame.shape[0]),
        steer_point,
        (0,0,255),
        4,
        tipLength=0.3
    )

    cv2.putText(output, f"Lane: {current_lane}", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    return output


Error: Cannot open video file


### کد نهایی با ورودی فیلم

In [1]:
import cv2
import numpy as np
import math

VIDEO_PATH = r"D:\FiraCUP\Soraat\PishVaDor\road.mp4"
DISPLAY_SCALE = 0.7

# ===== حافظه =====
prev_boxes_x = [None, None, None]  # چپ، وسط، راست
ALPHA = 0.25
MAX_JUMP = 80

# ===== مانع =====
CHANGE_LANE_THRESHOLD = 800
original_lane = None
lane_changed = False
obstacle_frame_count = 0
OBSTACLE_CONFIRMATION_FRAMES = 5
obstacle_distance_threshold = 100

# ================= توابع کمکی =================
def draw_square(img, center, size=14, color=(0,255,255)):
    x, y = center
    cv2.rectangle(img, (x-size, y-size),
                  (x+size, y+size), color, 2)

def smooth(prev, new):
    if prev is None:
        return new
    if abs(new - prev) > MAX_JUMP:
        return prev
    return int(ALPHA * new + (1 - ALPHA) * prev)

def angle_between_vertical(p1, p2):
    """محاسبه زاویه فلش نسبت به خط عمودی (محور Y معکوس)"""
    dx = p2[0] - p1[0]
    dy = p1[1] - p2[1]  # چون محور Y در تصویر برعکس است
    
    # محاسبه زاویه به درجه (محدوده -90 تا 90)
    angle_rad = math.atan2(dx, dy)
    angle_deg = math.degrees(angle_rad)
    
    # محدود کردن زاویه به بازه -90 تا 90 درجه
    if angle_deg > 90:
        angle_deg = 90
    elif angle_deg < -90:
        angle_deg = -90
    
    return angle_deg

# ================= تشخیص مانع =================
def detect_orange_obstacles(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower1 = np.array([5, 150, 150])
    upper1 = np.array([15, 255, 255])
    lower2 = np.array([170, 150, 150])
    upper2 = np.array([180, 255, 255])

    mask = cv2.bitwise_or(
        cv2.inRange(hsv, lower1, upper1),
        cv2.inRange(hsv, lower2, upper2)
    )

    kernel = np.ones((7,7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    return mask

def find_largest_obstacle(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None, 0
    
    min_area = 300
    valid_contours = [c for c in contours if cv2.contourArea(c) > min_area]
    
    if not valid_contours:
        return None, 0
    
    c = max(valid_contours, key=cv2.contourArea)
    return c, cv2.contourArea(c)

def is_obstacle_in_lane(cx, cy, lane, car_x, h):
    if cy < h * 0.5:
        return False
    
    if lane == "راست":
        return cx > car_x + 50 and abs(cx - car_x) < 300
    elif lane == "چپ":
        return cx < car_x - 50 and abs(cx - car_x) < 300
    
    return False

# ================= پردازش فریم =================
def process_frame(frame):
    global prev_boxes_x, original_lane, lane_changed, obstacle_frame_count

    h, w = frame.shape[:2]
    car_x = w // 2
    y_ref = int(h * 0.72)

    car_point = (car_x, h)
    vertical_ref = (car_x, int(h * 0.6))

    # ================== 1) تشخیص لاین ==================
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    mask = np.zeros_like(edges)
    roi = np.array([[(0,h),(w,h),(w,int(h*0.6)),(0,int(h*0.6))]])
    cv2.fillPoly(mask, roi, 255)
    edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(
        edges, 1, np.pi/180,
        threshold=50,
        minLineLength=50,
        maxLineGap=140
    )

    xs = []

    if lines is not None:
        for l in lines:
            x1,y1,x2,y2 = l[0]
            if abs(x2-x1) < 10:
                continue

            slope = (y2-y1)/(x2-x1)
            if abs(slope) < 0.3 or abs(slope) > 2:
                continue

            x_at_y = int(x1 + (y_ref-y1)*(x2-x1)/(y2-y1))
            if 0 < x_at_y < w:
                xs.append(x_at_y)

            cv2.line(frame,(x1,y1),(x2,y2),(100,100,255),1)

    if len(xs) < 10:
        cv2.putText(frame, "خطوط کافی نیست", (40,130),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        
        # چاپ در ترمینال حتی وقتی خطوط کافی نیست
        print("┌─────────────────────────────────────────────────────┐")
        print("│ Angle:    ---.--°  │ Lane:         نامشخص          │")
        print("│ Lane_Changed: False │ Obstacle:       No            │")
        print("└─────────────────────────────────────────────────────┘")
        
        return frame

    # ================== 2) مربع‌ها ==================
    hist = np.histogram(xs, bins=60, range=(0,w))[0]
    peaks = sorted(np.argsort(hist)[-3:])

    boxes_x = []
    for i, p in enumerate(peaks):
        x = int((p + 0.5) * (w / 60))
        x = smooth(prev_boxes_x[i], x)
        boxes_x.append(x)

    prev_boxes_x = boxes_x

    colors = [(255,0,0),(0,255,255),(0,255,0)]
    for i, x in enumerate(boxes_x):
        draw_square(frame, (x, y_ref), color=colors[i])

    # ================== 3) تشخیص لاین فعلی ==================
    left_count = sum(1 for x in boxes_x if x < car_x)
    right_count = sum(1 for x in boxes_x if x > car_x)

    if left_count == 2 and right_count == 1:
        lane = "راست"
        active_normal = (boxes_x[1], boxes_x[2])
    elif right_count == 2 and left_count == 1:
        lane = "چپ"
        active_normal = (boxes_x[0], boxes_x[1])
    else:
        lane = "نامشخص"
        active_normal = (boxes_x[0], boxes_x[2])

    if original_lane is None and lane != "نامشخص":
        original_lane = lane

    # ================== 4) تشخیص مانع ==================
    obstacle_detected = False
    obstacle_in_our_lane = False
    cx = None
    cy = None
    area = 0

    orange_mask = detect_orange_obstacles(frame)
    contour, area = find_largest_obstacle(orange_mask)

    if contour is not None and area > 0:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            
            cv2.circle(frame, (cx, cy), 8, (0,0,255), -1)
            cv2.putText(frame, f"({cx},{cy})", (cx+10, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
            
            obstacle_in_our_lane = is_obstacle_in_lane(cx, cy, lane, car_x, h)
            obstacle_detected = area >= CHANGE_LANE_THRESHOLD
            
            if obstacle_detected and obstacle_in_our_lane:
                obstacle_frame_count += 1
            else:
                obstacle_frame_count = max(0, obstacle_frame_count - 1)

    # ================== 5) منطق تغییر لاین ==================
    target_lane = lane
    steer_x = None

    obstacle_confirmed = obstacle_frame_count >= OBSTACLE_CONFIRMATION_FRAMES

    if not obstacle_confirmed or not obstacle_in_our_lane:
        if lane_changed:
            if original_lane == "راست":
                steer_x = (boxes_x[1] + boxes_x[2]) // 2
                target_lane = "راست (بازگشت)"
            elif original_lane == "چپ":
                steer_x = (boxes_x[0] + boxes_x[1]) // 2
                target_lane = "چپ (بازگشت)"
            lane_changed = False
            obstacle_frame_count = 0
        else:
            if lane == "راست":
                steer_x = (boxes_x[1] + boxes_x[2]) // 2
            elif lane == "چپ":
                steer_x = (boxes_x[0] + boxes_x[1]) // 2
            else:
                steer_x = (boxes_x[0] + boxes_x[2]) // 2
            target_lane = lane

    elif obstacle_confirmed and obstacle_in_our_lane and not lane_changed:
        if original_lane is None:
            original_lane = lane
        
        if lane == "راست":
            steer_x = (boxes_x[0] + boxes_x[1]) // 2
            target_lane = "چپ (تغییر)"
            lane_changed = True
        elif lane == "چپ":
            steer_x = (boxes_x[1] + boxes_x[2]) // 2
            target_lane = "راست (تغییر)"
            lane_changed = True

    # ================== 6) محاسبه فلش و زاویه ==================
    if steer_x is None:
        if lane == "راست":
            steer_x = (boxes_x[1] + boxes_x[2]) // 2
        elif lane == "چپ":
            steer_x = (boxes_x[0] + boxes_x[1]) // 2
        else:
            steer_x = (boxes_x[0] + boxes_x[2]) // 2
    
    steer_point = (steer_x, y_ref)

    # محاسبه زاویه
    angle = angle_between_vertical(car_point, steer_point)

    # ================== 7) رسم فلش و خطوط ==================
    if lane_changed:
        arrow_color = (0,255,255)
    else:
        arrow_color = (0,255,0)
    
    cv2.arrowedLine(frame, car_point, steer_point, arrow_color, 3)
    cv2.line(frame, car_point, vertical_ref, (255,255,255), 2)

    # ================== 8) نمایش اطلاعات روی تصویر ==================
    lane_color = (0,255,0) if not lane_changed else (0,255,255)
    cv2.putText(frame,
        f"Lane: {target_lane}",
        (40,50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9, lane_color, 2)

    cv2.putText(frame,
        f"Angle: {angle:+.1f}°",
        (40,90),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9, (255,255,255), 2)

    if obstacle_detected:
        obstacle_status = "در لاین" if obstacle_in_our_lane else "خارج لاین"
        status_color = (0,0,255) if obstacle_in_our_lane else (0,165,255)
        cv2.putText(frame,
            f"Obstacle: {obstacle_status}",
            (40,130),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7, status_color, 2)
        
        cv2.putText(frame,
            f"Area: {int(area)}, Frames: {obstacle_frame_count}/{OBSTACLE_CONFIRMATION_FRAMES}",
            (40,160),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7, status_color, 2)
        
        if obstacle_in_our_lane and lane_changed:
            cv2.putText(frame,
                "! تغییر لاین فعال",
                (40,190),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8, (0,255,255), 2)

    if original_lane:
        cv2.putText(frame,
            f"Original: {original_lane}",
            (w-250,50),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7, (200,200,200), 2)

    # ================== 9) چاپ اطلاعات در ترمینال ==================
    # قالب‌بندی زیبا برای ترمینال
    angle_str = f"{angle:+.2f}"
    
    # فرمت‌بندی لاین
    if lane == "نامشخص":
        lane_display = "نامشخص    "
    elif lane == "راست":
        lane_display = "راست      "
    else:
        lane_display = "چپ        "
    
    # فرمت‌بندی وضعیت مانع
    obstacle_display = "Yes      " if obstacle_detected else "No       "
    
    # چاپ در ترمینال با فرمت جدول
    print("┌─────────────────────────────────────────────────────┐")
    print(f"│ Angle:    {angle_str:>7}° │ Lane:         {lane_display}│")
    print(f"│ Lane_Changed: {str(lane_changed):<5} │ Obstacle:       {obstacle_display}│")
    print("└─────────────────────────────────────────────────────┘")

    return frame

# ================= اجرای اصلی =================
cap = cv2.VideoCapture(VIDEO_PATH)
print("\n" + "="*60)
print("🎥 سیستم تشخیص لاین و مانع - نسخه نهایی")
print("="*60)
print("📊 اطلاعات خروجی در ترمینال:")
print("  - Angle: زاویه فلش نسبت به خط عمودی (-90° تا +90°)")
print("  - Lane: لاین فعلی (چپ / راست / نامشخص)")
print("  - Lane_Changed: وضعیت تغییر لاین (True/False)")
print("  - Obstacle: وجود مانع در تصویر (Yes/No)")
print("="*60)
print("🎮 کلیدهای کنترل:")
print("  - q: خروج")
print("  - r: ریست سیستم")
print("="*60 + "\n")

while True:
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    processed_frame = process_frame(frame)
    display_frame = cv2.resize(processed_frame, None,
                               fx=DISPLAY_SCALE,
                               fy=DISPLAY_SCALE)

    cv2.imshow("Combined Lane & Obstacle Detection", display_frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("\n" + "="*60)
        print("👋 برنامه خاتمه یافت")
        print("="*60)
        break
    elif key == ord('r'):
        original_lane = None
        lane_changed = False
        obstacle_frame_count = 0
        prev_boxes_x = [None, None, None]
        print("\n" + "="*60)
        print("🔄 سیستم ریست شد")
        print("="*60)

cap.release()
cv2.destroyAllWindows()


🎥 سیستم تشخیص لاین و مانع - نسخه نهایی
📊 اطلاعات خروجی در ترمینال:
  - Angle: زاویه فلش نسبت به خط عمودی (-90° تا +90°)
  - Lane: لاین فعلی (چپ / راست / نامشخص)
  - Lane_Changed: وضعیت تغییر لاین (True/False)
  - Obstacle: وجود مانع در تصویر (Yes/No)
🎮 کلیدهای کنترل:
  - q: خروج
  - r: ریست سیستم

┌─────────────────────────────────────────────────────┐
│ Angle:      +6.03° │ Lane:         راست      │
│ Lane_Changed: False │ Obstacle:       No       │
└─────────────────────────────────────────────────────┘
┌─────────────────────────────────────────────────────┐
│ Angle:      +6.78° │ Lane:         راست      │
│ Lane_Changed: False │ Obstacle:       No       │
└─────────────────────────────────────────────────────┘
┌─────────────────────────────────────────────────────┐
│ Angle:      +7.33° │ Lane:         راست      │
│ Lane_Changed: False │ Obstacle:       No       │
└─────────────────────────────────────────────────────┘
┌─────────────────────────────────────────────────────┐
│ Angle:

KeyboardInterrupt: 

### کد نهایی با ورودی زنده از دوربین

In [3]:
import cv2
import numpy as np
import math
from picamera2 import Picamera2
from libcamera import Transform

DISPLAY_SCALE = 0.7

# ===== حافظه =====
prev_boxes_x = [None, None, None]  # چپ، وسط، راست
ALPHA = 0.25
MAX_JUMP = 80

# ===== مانع =====
CHANGE_LANE_THRESHOLD = 800
original_lane = None
lane_changed = False
obstacle_frame_count = 0
OBSTACLE_CONFIRMATION_FRAMES = 5
obstacle_distance_threshold = 100

# ================= توابع کمکی =================
def draw_square(img, center, size=14, color=(0,255,255)):
    x, y = center
    cv2.rectangle(img, (x-size, y-size),
                  (x+size, y+size), color, 2)

def smooth(prev, new):
    if prev is None:
        return new
    if abs(new - prev) > MAX_JUMP:
        return prev
    return int(ALPHA * new + (1 - ALPHA) * prev)

def angle_between_vertical(p1, p2):
    """محاسبه زاویه فلش نسبت به خط عمودی (محور Y معکوس)"""
    dx = p2[0] - p1[0]
    dy = p1[1] - p2[1]  # چون محور Y در تصویر برعکس است
    
    # محاسبه زاویه به درجه (محدوده -90 تا 90)
    angle_rad = math.atan2(dx, dy)
    angle_deg = math.degrees(angle_rad)
    
    # محدود کردن زاویه به بازه -90 تا 90 درجه
    if angle_deg > 90:
        angle_deg = 90
    elif angle_deg < -90:
        angle_deg = -90
    
    return angle_deg

# ================= تشخیص مانع =================
def detect_orange_obstacles(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower1 = np.array([5, 150, 150])
    upper1 = np.array([15, 255, 255])
    lower2 = np.array([170, 150, 150])
    upper2 = np.array([180, 255, 255])

    mask = cv2.bitwise_or(
        cv2.inRange(hsv, lower1, upper1),
        cv2.inRange(hsv, lower2, upper2)
    )

    kernel = np.ones((7,7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    return mask

def find_largest_obstacle(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None, 0
    
    min_area = 300
    valid_contours = [c for c in contours if cv2.contourArea(c) > min_area]
    
    if not valid_contours:
        return None, 0
    
    c = max(valid_contours, key=cv2.contourArea)
    return c, cv2.contourArea(c)

def is_obstacle_in_lane(cx, cy, lane, car_x, h):
    if cy < h * 0.5:
        return False
    
    if lane == "راست":
        return cx > car_x + 50 and abs(cx - car_x) < 300
    elif lane == "چپ":
        return cx < car_x - 50 and abs(cx - car_x) < 300
    
    return False

# ================= پردازش فریم =================
def process_frame(frame):
    global prev_boxes_x, original_lane, lane_changed, obstacle_frame_count

    h, w = frame.shape[:2]
    car_x = w // 2
    y_ref = int(h * 0.72)

    car_point = (car_x, h)
    vertical_ref = (car_x, int(h * 0.6))

    # ================== 1) تشخیص لاین ==================
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    mask = np.zeros_like(edges)
    roi = np.array([[(0,h),(w,h),(w,int(h*0.6)),(0,int(h*0.6))]])
    cv2.fillPoly(mask, roi, 255)
    edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(
        edges, 1, np.pi/180,
        threshold=50,
        minLineLength=50,
        maxLineGap=140
    )

    xs = []

    if lines is not None:
        for l in lines:
            x1,y1,x2,y2 = l[0]
            if abs(x2-x1) < 10:
                continue

            slope = (y2-y1)/(x2-x1)
            if abs(slope) < 0.3 or abs(slope) > 2:
                continue

            x_at_y = int(x1 + (y_ref-y1)*(x2-x1)/(y2-y1))
            if 0 < x_at_y < w:
                xs.append(x_at_y)

            cv2.line(frame,(x1,y1),(x2,y2),(100,100,255),1)

    if len(xs) < 10:
        cv2.putText(frame, "خطوط کافی نیست", (40,130),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        
        # چاپ در ترمینال حتی وقتی خطوط کافی نیست
        print("┌─────────────────────────────────────────────────────┐")
        print("│ Angle:    ---.--°  │ Lane:         نامشخص          │")
        print("│ Lane_Changed: False │ Obstacle:       No            │")
        print("└─────────────────────────────────────────────────────┘")
        
        return frame

    # ================== 2) مربع‌ها ==================
    hist = np.histogram(xs, bins=60, range=(0,w))[0]
    peaks = sorted(np.argsort(hist)[-3:])

    boxes_x = []
    for i, p in enumerate(peaks):
        x = int((p + 0.5) * (w / 60))
        x = smooth(prev_boxes_x[i], x)
        boxes_x.append(x)

    prev_boxes_x = boxes_x

    colors = [(255,0,0),(0,255,255),(0,255,0)]
    for i, x in enumerate(boxes_x):
        draw_square(frame, (x, y_ref), color=colors[i])

    # ================== 3) تشخیص لاین فعلی ==================
    left_count = sum(1 for x in boxes_x if x < car_x)
    right_count = sum(1 for x in boxes_x if x > car_x)

    if left_count == 2 and right_count == 1:
        lane = "راست"
        active_normal = (boxes_x[1], boxes_x[2])
    elif right_count == 2 and left_count == 1:
        lane = "چپ"
        active_normal = (boxes_x[0], boxes_x[1])
    else:
        lane = "نامشخص"
        active_normal = (boxes_x[0], boxes_x[2])

    if original_lane is None and lane != "نامشخص":
        original_lane = lane

    # ================== 4) تشخیص مانع ==================
    obstacle_detected = False
    obstacle_in_our_lane = False
    cx = None
    cy = None
    area = 0

    orange_mask = detect_orange_obstacles(frame)
    contour, area = find_largest_obstacle(orange_mask)

    if contour is not None and area > 0:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            
            cv2.circle(frame, (cx, cy), 8, (0,0,255), -1)
            cv2.putText(frame, f"({cx},{cy})", (cx+10, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
            
            obstacle_in_our_lane = is_obstacle_in_lane(cx, cy, lane, car_x, h)
            obstacle_detected = area >= CHANGE_LANE_THRESHOLD
            
            if obstacle_detected and obstacle_in_our_lane:
                obstacle_frame_count += 1
            else:
                obstacle_frame_count = max(0, obstacle_frame_count - 1)

    # ================== 5) منطق تغییر لاین ==================
    target_lane = lane
    steer_x = None

    obstacle_confirmed = obstacle_frame_count >= OBSTACLE_CONFIRMATION_FRAMES

    if not obstacle_confirmed or not obstacle_in_our_lane:
        if lane_changed:
            if original_lane == "راست":
                steer_x = (boxes_x[1] + boxes_x[2]) // 2
                target_lane = "راست (بازگشت)"
            elif original_lane == "چپ":
                steer_x = (boxes_x[0] + boxes_x[1]) // 2
                target_lane = "چپ (بازگشت)"
            lane_changed = False
            obstacle_frame_count = 0
        else:
            if lane == "راست":
                steer_x = (boxes_x[1] + boxes_x[2]) // 2
            elif lane == "چپ":
                steer_x = (boxes_x[0] + boxes_x[1]) // 2
            else:
                steer_x = (boxes_x[0] + boxes_x[2]) // 2
            target_lane = lane

    elif obstacle_confirmed and obstacle_in_our_lane and not lane_changed:
        if original_lane is None:
            original_lane = lane
        
        if lane == "راست":
            steer_x = (boxes_x[0] + boxes_x[1]) // 2
            target_lane = "چپ (تغییر)"
            lane_changed = True
        elif lane == "چپ":
            steer_x = (boxes_x[1] + boxes_x[2]) // 2
            target_lane = "راست (تغییر)"
            lane_changed = True

    # ================== 6) محاسبه فلش و زاویه ==================
    if steer_x is None:
        if lane == "راست":
            steer_x = (boxes_x[1] + boxes_x[2]) // 2
        elif lane == "چپ":
            steer_x = (boxes_x[0] + boxes_x[1]) // 2
        else:
            steer_x = (boxes_x[0] + boxes_x[2]) // 2
    
    steer_point = (steer_x, y_ref)

    # محاسبه زاویه
    angle = angle_between_vertical(car_point, steer_point)

    # ================== 7) رسم فلش و خطوط ==================
    if lane_changed:
        arrow_color = (0,255,255)
    else:
        arrow_color = (0,255,0)
    
    cv2.arrowedLine(frame, car_point, steer_point, arrow_color, 3)
    cv2.line(frame, car_point, vertical_ref, (255,255,255), 2)

    # ================== 8) نمایش اطلاعات روی تصویر ==================
    lane_color = (0,255,0) if not lane_changed else (0,255,255)
    cv2.putText(frame,
        f"Lane: {target_lane}",
        (40,50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9, lane_color, 2)

    cv2.putText(frame,
        f"Angle: {angle:+.1f}°",
        (40,90),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9, (255,255,255), 2)

    if obstacle_detected:
        obstacle_status = "در لاین" if obstacle_in_our_lane else "خارج لاین"
        status_color = (0,0,255) if obstacle_in_our_lane else (0,165,255)
        cv2.putText(frame,
            f"Obstacle: {obstacle_status}",
            (40,130),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7, status_color, 2)
        
        cv2.putText(frame,
            f"Area: {int(area)}, Frames: {obstacle_frame_count}/{OBSTACLE_CONFIRMATION_FRAMES}",
            (40,160),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7, status_color, 2)
        
        if obstacle_in_our_lane and lane_changed:
            cv2.putText(frame,
                "! تغییر لاین فعال",
                (40,190),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8, (0,255,255), 2)

    if original_lane:
        cv2.putText(frame,
            f"Original: {original_lane}",
            (w-250,50),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7, (200,200,200), 2)

    # ================== 9) چاپ اطلاعات در ترمینال ==================
    # قالب‌بندی زیبا برای ترمینال
    angle_str = f"{angle:+.2f}"
    
    # فرمت‌بندی لاین
    if lane == "نامشخص":
        lane_display = "نامشخص    "
    elif lane == "راست":
        lane_display = "راست      "
    else:
        lane_display = "چپ        "
    
    # فرمت‌بندی وضعیت مانع
    obstacle_display = "Yes      " if obstacle_detected else "No       "
    
    # چاپ در ترمینال با فرمت جدول
    print("┌─────────────────────────────────────────────────────┐")
    print(f"│ Angle:    {angle_str:>7}° │ Lane:         {lane_display}│")
    print(f"│ Lane_Changed: {str(lane_changed):<5} │ Obstacle:       {obstacle_display}│")
    print("└─────────────────────────────────────────────────────┘")

    return frame

# ================= اجرای اصلی =================
# راه‌اندازی دوربین رزپری
print("\n" + "="*60)
print("🎥 سیستم تشخیص لاین و مانع - نسخه نهایی (Raspberry Pi)")
print("="*60)

# راه‌اندازی Picamera2
picam2 = Picamera2()

# پیکربندی دوربین
config = picam2.create_preview_configuration(
    main={"size": (640, 480)},  # رزولوشن مناسب
    # اگر تصویر وارونه است، یکی از خطوط زیر را آنکامنت کنید:
    # transform=Transform(hflip=True),  # برای چرخش افقی
    # transform=Transform(vflip=True),  # برای چرخش عمودی
    # transform=Transform(hflip=True, vflip=True)  # برای چرخش 180 درجه
)
picam2.configure(config)
picam2.start()

print("📷 دوربین رزپری راه‌اندازی شد")
print("📊 اطلاعات خروجی در ترمینال:")
print("  - Angle: زاویه فلش نسبت به خط عمودی (-90° تا +90°)")
print("  - Lane: لاین فعلی (چپ / راست / نامشخص)")
print("  - Lane_Changed: وضعیت تغییر لاین (True/False)")
print("  - Obstacle: وجود مانع در تصویر (Yes/No)")
print("="*60)
print("🎮 کلیدهای کنترل:")
print("  - q: خروج")
print("  - r: ریست سیستم")
print("="*60 + "\n")

while True:
    # گرفتن فریم از دوربین رزپری
    frame = picam2.capture_array()
    
    # تبدیل از RGB به BGR (OpenCV از BGR استفاده می‌کند)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # پردازش فریم (همان منطق قبلی)
    processed_frame = process_frame(frame)
    display_frame = cv2.resize(processed_frame, None,
                               fx=DISPLAY_SCALE,
                               fy=DISPLAY_SCALE)

    cv2.imshow("Combined Lane & Obstacle Detection (Raspberry Pi)", display_frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("\n" + "="*60)
        print("👋 برنامه خاتمه یافت")
        print("="*60)
        break
    elif key == ord('r'):
        original_lane = None
        lane_changed = False
        obstacle_frame_count = 0
        prev_boxes_x = [None, None, None]
        print("\n" + "="*60)
        print("🔄 سیستم ریست شد")
        print("="*60)

# پاکسازی
picam2.stop()
cv2.destroyAllWindows()
print("📷 دوربین متوقف شد")

ModuleNotFoundError: No module named 'picamera2'